<a href="https://colab.research.google.com/github/Deok-Hun/Aiffel_Kun/blob/master/EX_13_Movielens_%EC%98%81%ED%99%94_%EC%B6%94%EC%B2%9C_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보겠습니다.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터입니다.

    * 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
    * 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
    * 별점을 시청횟수로 해석해서 생각하겠습니다.
    * 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.

In [ ]:
import numpy as np
import scipy
import implicit

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


# 데이터 준비와 전처리

In [ ]:
import os
import pandas as pd
rating_file_path = os.getenv('HOME')+'/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names = ratings_cols, engine='python', encoding = "ISO-8859-1")
original_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
# 3점 이상만 남긴다.
ratings = ratings[ratings['ratings'] >= 3]
filtered_data_size = len(ratings)

print(f'original_data_size: {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / original_data_size:.2%}')

original_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [ ]:
# ratings 컬럼의 이름을 counts로 바꾼다.
ratings.rename(columns = {'ratings' : 'counts'}, inplace = True)

In [ ]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [ ]:
# 영화 제목을 보기 위해 메타 데이터를 읽어온다.
movie_file_path = os.getenv("HOME")+'/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv(movie_file_path, sep = '::', names = cols, engine = 'python', encoding = 'ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
# rating 데이터프레임과 movie 데이터프레임을 merge 한다.
ratings = pd.merge(ratings, movies, on ='movie_id', how='left')
ratings

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy
...,...,...,...,...,...,...
836473,6040,1090,3,956715518,Platoon (1986),Drama|War
836474,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama
836476,6040,1096,4,956715648,Sophie's Choice (1982),Drama


In [ ]:
ratings['counts'].unique()

array([5, 3, 4])

In [ ]:
# 검색을 쉽게 하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
ratings['title'] = ratings['title'].str.lower()
ratings.head(5)

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),Drama
1,1,661,3,978302109,james and the giant peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,my fair lady (1964),Musical|Romance
3,1,3408,4,978300275,erin brockovich (2000),Drama
4,1,2355,5,978824291,"bug's life, a (1998)",Animation|Children's|Comedy


# 분석해 봅시다.
    * ratings에 있는 유니크한 영화 개수
    * ratings에 있는 유니크한 사용자 수
    * 가장 인기 있는 영화 30개(인기순)

In [ ]:
# ratings에 있는 유니크한 영화 개수
ratings['title'].nunique()

3628

In [ ]:
# ratings에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [ ]:
pd.set_option('display.max_row',100)

In [ ]:
# 가장 인기 있는 영화 30개(인기수)
movie_counts = ratings.groupby('title')['counts'].count()
movie_counts.sort_values(ascending=False).head(100)

title
american beauty (1999)                                                         3211
star wars: episode iv - a new hope (1977)                                      2910
star wars: episode v - the empire strikes back (1980)                          2885
star wars: episode vi - return of the jedi (1983)                              2716
saving private ryan (1998)                                                     2561
terminator 2: judgment day (1991)                                              2509
silence of the lambs, the (1991)                                               2498
raiders of the lost ark (1981)                                                 2473
back to the future (1985)                                                      2460
matrix, the (1999)                                                             2434
jurassic park (1993)                                                           2413
sixth sense, the (1999)                                               

# 내가 선호하는 영화를 5가지 골라서 ratings에 추가

In [ ]:
ratings[ratings["title"]== 'toy story (1995)'].head(1)

,user_id,movie_id,counts,timestamp,title,genre
40,1,1,5,978824268,toy story (1995),Animation|Children's|Comedy


In [ ]:
ratings[ratings["title"]== 'mission: impossible (1996)'].head(1)

,user_id,movie_id,counts,timestamp,title,genre
58,2,648,4,978299913,mission: impossible (1996),Action|Adventure|Mystery


In [ ]:
ratings[ratings["title"]== 'die hard (1988)'].head(1)

,user_id,movie_id,counts,timestamp,title,genre
219,4,1036,4,978294282,die hard (1988),Action|Thriller


In [ ]:
ratings[ratings["title"]== 'forrest gump (1994)'].head(1)

,user_id,movie_id,counts,timestamp,title,genre
166,2,356,5,978299686,forrest gump (1994),Comedy|Romance|War


In [ ]:
ratings[ratings["title"]== 'matrix, the (1999)'].head(1)

,user_id,movie_id,counts,timestamp,title,genre
127,2,2571,4,978299773,"matrix, the (1999)",Action|Sci-Fi|Thriller


In [ ]:
my_favorite = ['toy story (1995)', 'mission: impossible (1996)', 'die hard (1988)', 'forrest gump (1994)', 'matrix, the (1999)']

# 'user_id'는 6041 로 위 영화의 별점 5점을 주었다고 가정
my_movielist = pd.DataFrame({'user_id':['kun']*5, 'movie_id':['1','648','1036','356','2571'],'title': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':['kun']})['user_id'].any():
    ratings = ratings.append(my_movielist)
    
ratings.tail(10)

,user_id,movie_id,counts,timestamp,title,genre
836473,6040,1090,3,956715518.0,platoon (1986),Drama|War
836474,6040,1094,5,956704887.0,"crying game, the (1992)",Drama|Romance|War
836475,6040,562,5,956704746.0,welcome to the dollhouse (1995),Comedy|Drama
836476,6040,1096,4,956715648.0,sophie's choice (1982),Drama
836477,6040,1097,4,956715569.0,e.t. the extra-terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,kun,1,5,NaN,toy story (1995),NaN
1,kun,648,5,NaN,mission: impossible (1996),NaN
2,kun,1036,5,NaN,die hard (1988),NaN
3,kun,356,5,NaN,forrest gump (1994),NaN
4,kun,2571,5,NaN,"matrix, the (1999)",NaN


In [ ]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
title_unique = ratings['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [ ]:
print(user_to_idx['kun'])
print(title_to_idx['platoon (1986)'])

6039
1030


In [ ]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # ratings['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# title_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = ratings['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(ratings):
    print('title column indexing OK!!')
    ratings['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

ratings

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,counts,timestamp,title,genre
0,0,1193,5,978300760.0,0,Drama
1,0,661,3,978302109.0,1,Animation|Children's|Musical
2,0,914,3,978301968.0,2,Musical|Romance
3,0,3408,4,978300275.0,3,Drama
4,0,2355,5,978824291.0,4,Animation|Children's|Comedy
...,...,...,...,...,...,...
0,6039,1,5,NaN,40,NaN
1,6039,648,5,NaN,58,NaN
2,6039,1036,5,NaN,194,NaN
3,6039,356,5,NaN,160,NaN


In [ ]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'counts', 'title']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,counts,title
0,0,5,0
1,0,3,1
2,0,3,2
3,0,4,3
4,0,5,4
5,0,3,5
6,0,5,6
7,0,5,7
8,0,4,8
9,0,4,9


In [ ]:
ratings=ratings.astype('int64')

In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836483 entries, 0 to 4
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype
---  ------   --------------   -----
 0   user_id  836483 non-null  int64
 1   counts   836483 non-null  int64
 2   title    836483 non-null  int64
dtypes: int64(3)
memory usage: 25.5 MB


# CSR Matrix를 직접 만들어 봅시다.

In [ ]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_title = ratings['title'].nunique()

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [ ]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [ ]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [ ]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [ ]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

# 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

## 내가 실제 선호하는 영화와 모델이 예측한 그 영화의 선호도 확인 

In [ ]:
kun, toy_story = user_to_idx['kun'], title_to_idx['toy story (1995)']
kun_vector, toy_story_vector = als_model.user_factors[kun], als_model.item_factors[toy_story]

In [ ]:
kun_vector

array([ 0.48564586, -0.5140683 ,  0.26029524,  0.34289983, -0.8122591 ,
        0.00455766, -1.0293972 ,  0.48401913, -0.20149885, -0.3162134 ,
       -0.00915215, -0.26099473,  0.5659091 ,  0.7363566 ,  0.7622368 ,
        0.07903162, -0.65878665,  0.41252163,  0.41712883,  0.1424305 ,
        0.15777569, -0.6695809 , -0.5546065 , -1.1821939 ,  0.47202247,
       -0.15125035,  0.19178118,  1.1179024 ,  0.37866884, -0.18327233,
       -0.56343025, -0.16458343,  0.82297766, -0.94872904, -0.23546895,
        0.31812012,  0.3037962 , -0.50011724,  0.1983121 , -0.11627788,
       -0.19634326, -0.01270496, -0.18913846,  0.31768292, -0.03908355,
       -0.9766641 , -0.13254495, -0.4619123 , -0.15823138,  1.2658309 ,
        0.31832328,  0.6005992 , -0.25277257, -0.8014007 , -0.4269022 ,
        0.02912686, -0.03519986,  0.73600155,  0.19287813,  0.34162748,
        0.27048203, -0.15468872,  1.0710722 ,  0.32427338, -0.68114   ,
        0.24978618, -0.22112958,  0.16557261,  0.16663173, -0.19

In [ ]:
toy_story_vector

array([ 0.0313688 , -0.01006434,  0.03413722, -0.01002307, -0.00891121,
       -0.00369829, -0.00495819,  0.01281466, -0.01187521,  0.00709381,
       -0.01585265, -0.02411765,  0.00708074,  0.01641825,  0.02017337,
       -0.00162592, -0.00671582,  0.00746346, -0.00796722, -0.00635411,
        0.0094764 , -0.00099062, -0.00164766, -0.04653632,  0.01924358,
        0.0033    ,  0.00514898,  0.04189775,  0.01925165,  0.01163242,
       -0.02346786, -0.00893929,  0.03144253,  0.03379799, -0.00320556,
        0.01918787, -0.01118597, -0.03500988, -0.02718343,  0.00505196,
       -0.01366804, -0.00242427,  0.00560574,  0.02339277,  0.02102142,
       -0.01231173,  0.00693075, -0.01022029,  0.00091066,  0.03078254,
        0.01260038,  0.01360719,  0.02932178, -0.00904928, -0.00526981,
        0.0192821 ,  0.0259833 , -0.00248732, -0.00394403, -0.00799586,
        0.01474291,  0.02349219,  0.02234766,  0.02796217, -0.01941177,
        0.00198478, -0.029088  , -0.0047425 ,  0.00820061, -0.03

In [ ]:
# kun과 toy_story를 내적하는 코드
np.dot(kun_vector, toy_story_vector)

0.5450615

## 선호하는 모델 외 어느 한 영화가 나의 선호도 관계확인

In [ ]:
# kun과 'sixth sense, the (1999)'을 내적하는 코드 
sixth_sense = title_to_idx['sixth sense, the (1999)']
sixth_sense_vector = als_model.item_factors[sixth_sense]
np.dot(kun_vector, sixth_sense_vector)

0.2684208

# 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

## 내가 좋아하는 영화와 비슷한 영화

In [ ]:
my_favorite

['toy story (1995)',
 'mission: impossible (1996)',
 'die hard (1988)',
 'forrest gump (1994)',
 'matrix, the (1999)']

In [ ]:
favorite_movie = 'matrix, the (1999)'
title_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(title_id, N = 15)
similar_movie

[(124, 1.0000001),
 (92, 0.7667988),
 (62, 0.658755),
 (141, 0.5950956),
 (145, 0.57889485),
 (200, 0.57228196),
 (107, 0.5558865),
 (375, 0.52737427),
 (175, 0.49249583),
 (317, 0.46871826),
 (75, 0.4460868),
 (431, 0.40040025),
 (3457, 0.39928803),
 (44, 0.38682163),
 (117, 0.3820667)]

In [ ]:
# title의 id(인덱스)를 다시 title의 이름으로 매핑한다.
# title_to_idx를 뒤집어, index로부터 title 이름을 얻는 dict를 생성
def get_similar_movie(movie_name: str):
    title_id = title_to_idx[movie_name]
    similar_movie = als_model.similar_items(title_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [ ]:
get_similar_movie('matrix, the (1999)')

['matrix, the (1999)',
 'terminator 2: judgment day (1991)',
 'total recall (1990)',
 'fugitive, the (1993)',
 'fifth element, the (1997)',
 'terminator, the (1984)',
 'jurassic park (1993)',
 'face/off (1997)',
 'men in black (1997)',
 'twelve monkeys (1995)']

## 내가 선호하지 않는 영화 (sixth sense)와 비슷한 영화

In [ ]:
get_similar_movie('sixth sense, the (1999)')

['sixth sense, the (1999)',
 'silence of the lambs, the (1991)',
 'usual suspects, the (1995)',
 'ghostbusters (1984)',
 'fight club (1999)',
 'seven (se7en) (1995)',
 'being john malkovich (1999)',
 'sleepy hollow (1999)',
 'bone collector, the (1999)',
 'fugitive, the (1993)']

# 내가 가장 좋아할 만한 영화들을 추천받아 본다.

In [ ]:
user = user_to_idx['kun']

# recommend 에서는 user*item CSR Matrix 를 받는다.
movie_recommend = als_model.recommend(user, csr_data, N = 20, filter_already_liked_items=True)
movie_recommend

[(141, 0.4644236),
 (92, 0.44193608),
 (50, 0.38353476),
 (110, 0.33848652),
 (200, 0.32949868),
 (75, 0.3168424),
 (64, 0.30793032),
 (107, 0.2945409),
 (236, 0.2861783),
 (48, 0.27761862),
 (38, 0.2684208),
 (117, 0.26287282),
 (62, 0.262338),
 (172, 0.2430471),
 (33, 0.23856108),
 (4, 0.22880273),
 (330, 0.2248289),
 (175, 0.2234117),
 (120, 0.20911731),
 (67, 0.1997397)]

In [ ]:
[idx_to_title[i[0]] for i in movie_recommend]

['fugitive, the (1993)',
 'terminator 2: judgment day (1991)',
 'toy story 2 (1999)',
 'groundhog day (1993)',
 'terminator, the (1984)',
 'hunt for red october, the (1990)',
 'star wars: episode vi - return of the jedi (1983)',
 'jurassic park (1993)',
 'speed (1994)',
 'saving private ryan (1998)',
 'sixth sense, the (1999)',
 'star wars: episode v - the empire strikes back (1980)',
 'total recall (1990)',
 'indiana jones and the last crusade (1989)',
 'aladdin (1992)',
 "bug's life, a (1998)",
 'lion king, the (1994)',
 'men in black (1997)',
 'raiders of the lost ark (1981)',
 'gladiator (2000)']

# 결론

  * 모델이 추천해주는 정확도는 내가 선호하는 영화리스트와 유사했다. 다만, 내가 선호하는 영화들과 나의(user)와의 관계는 0.545 정도로 생각보다 낮음을 알 수 있다. 예제에서는 factor를 늘리거나 iterations을 늘리는 방향을 암시하고 있다. 하지만, 요인을 늘리는 것과 iteration을 늘리는 방법에는 overfitting 의 문제가 있을 수 있다. 물론 현재 데이터에서는 테스트 모델이 없으므로 문제가 발생하지 않지만, 실전에서는 이부분을 염두에 두어야 할 것 같다.
  * 현 학습 모델에서는 협업 필터링으로 아이템 평점과 평점의 개수를 바탕으로 사용자간 또는 아이템간 유사성을 파악하였다. 하지만, 실습데이터에서 장르 컬럼을 이용하여 콘텐츠 기반 필터링을 통한 추천 시스템의 결과도 궁금해 졌다.